In [ ]:
import functions as utils
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
n = 1000
β = 1 
π = 1
x, y, Z = utils.dgp(n, β, π)

b0 = np.linspace(0.8, 1.2, 100)
p = np.array([utils.hansen(x, y, Z, b) for b in b0])

fig, ax = plt.subplots() 
ax.scatter(b0, p)
ax.set_xlabel('b0')
ax.set_ylabel('p-val')
